# Utilities

In [1]:
import sys, threading

sys.setrecursionlimit(10**7)
threading.stack_size(2**27)

def ConvertToInt(message_str: str) -> int:
  res = 0
  for i in range(len(message_str)):
    res = res * 256 + ord(message_str[i])
  return res

def ConvertToStr(n: int) -> str:
    res = ""
    while n > 0:
        res += chr(n % 256)
        n //= 256
    return res[::-1]

print(ConvertToStr(ConvertToInt('Number Theory'))) # to test the code


Number Theory


# Mathimatical Functions:

In [2]:
def GCD(a:int, b:int) -> int:
  if b == 0:
    return a
  return GCD(b, a % b)

def ExtendedEuclid(a:int, b:int) -> tuple[int, int]:
    if b == 0:
        return (1, 0)
    (x, y) = ExtendedEuclid(b, a % b)
    k = a // b
    return (y, x - k * y)

def PowMod(a:int, n:int, mod:int) -> int:
    """
    This is an R2L recursive implementation that works for large integers
    """
    if n == 0:
        return 1 % mod
    elif n == 1:
        return a % mod
    else:
        b = PowMod(a, n // 2, mod)
        b = b * b % mod
        if n % 2 == 0:
          return b
        else:
          return b * a % mod

def InvertModulo(a: int, n: int) -> int:
    (b, _) = ExtendedEuclid(a, n)
    if b < 0:
        b = (b % n + n) % n # as we don't want -ve integers
    return b

def chineseRemainderTheorem(n1: int, r1: int, n2: int, r2: int) -> int:
    """
    Uses ExtendedEuclid to find inverses
    """
    (x, y) = ExtendedEuclid(n1, n2)
    m = n1 * n2
    n = r2 * x * n1 + r1 * y * n2
    return (n % m + m) % m


# Question 1, simple attack

In [3]:
def Encrypt(m, n, e):
    return PowMod(ConvertToInt(m), e, n)
    
def Decrypt(c, p, q, e): 
  phi = (p-1)*(q-1)
  d = InvertModulo(e, phi)
  return ConvertToStr(PowMod(c, d, p * q )) 
  

# question 
p = 1000000007
q = 1000000009
exponent = 23917
modulo = p * q
ciphertext = Encrypt("attack", modulo, exponent)
message = Decrypt(ciphertext, p, q, exponent)
print(message)

attack


# Question 2, small primes

In [4]:
def DecipherSimple(c, n, e, potential_messages):
  for i in range(len(potential_messages)):
    if c == Encrypt(potential_messages[i], n, e):
      return potential_messages[i]
  return "Failed to decrypt the message"

# question 
modulo = 101
exponent = 12
ciphertext = Encrypt("attack", modulo, exponent)
print(DecipherSimple(ciphertext, modulo, exponent, ["attack", "don't attack", "wait"]))

attack


# Question 3, small difference between p and q

In [5]:
LIMIT = 1000000
def DecipherSmallPrime(c, n, e):
  if n % 2 == 0:
    smallerPrime = 2
    biggerPrime = n // 2
    return Decrypt(c, smallerPrime, biggerPrime, e) 
  # try all prime up to LIMIT as divisor of the public key n
  for i in range(3, LIMIT, 2):
    if n % i == 0:
      smallerPrime = i
      biggerPrime = n // i
    # factorize n and decrypt the cipher
      return Decrypt(c, smallerPrime, biggerPrime, e) 
  return "Failed to decrypt the message"
  
# question 
modulo = 101 * 18298970732541109011012304219376080251334480295537316123696052970419466495220522723330315111017831737980079504337868198011077274303193766040393009648852841770668239779097280026631944319501437547002412556176186750790476901358334138818777298389724049250700606462316428106882097210008142941838672676714188593227684360287806974345181893018133710957167334490627178666071809992955566020058374505477745993383434501768887090900283569055646901291270870833498474402084748161755197005050874785474707550376333429671113753137201128897550014524209754619355308207537703754006699795711188492048286436285518105948050401762394690148387
exponent = 239
ciphertext = Encrypt("attack", modulo, exponent)
print(DecipherSmallPrime(ciphertext, modulo, exponent))


attack


# Question 4, small difference between p and q

In [6]:
DIFF = 5000
import numpy as np
def DecipherSmallDiff(c, n, e):
	upperBound = int(round(np.sqrt(n)))
	bottomBound = upperBound - DIFF
	# Try all integers between root(n) - r and root(n) as divisors of n 
	for i in range(bottomBound, upperBound+1):
		if n % i == 0:
			smallerPrime = i
			biggerPrime = n // i
			return Decrypt(c, smallerPrime, biggerPrime, e)
	return "Failed to decrypt the message"

# question 
p = 1000000007
q = 1000000009
n = p * q
e = 239
ciphertext = Encrypt("attack", n, e)
message = DecipherSmallDiff(ciphertext, n, e)
print(message)

attack


# Question 5, insufficient randomness

In [7]:
def DecipherCommonDivisor(c1, n1, e1, c2, n2, e2):
  commonPrime = GCD(n1, n2)
  q1 = n1 // commonPrime
  q2 = n2 // commonPrime
  msg1 = Decrypt(c1, commonPrime, q1, e1)
  msg2 = Decrypt(c2, commonPrime, q2, e2)
  return (msg1, msg2)


# question 
p = 101
q1 = 18298970732541109011012304219376080251334480295537316123696052970419466495220522723330315111017831737980079504337868198011077274303193766040393009648852841770668239779097280026631944319501437547002412556176186750790476901358334138818777298389724049250700606462316428106882097210008142941838672676714188593227684360287806974345181893018133710957167334490627178666071809992955566020058374505477745993383434501768887090900283569055646901291270870833498474402084748161755197005050874785474707550376333429671113753137201128897550014524209754619355308207537703754006699795711188492048286436285518105948050401762394690148387
q2 = 1000000007
first_modulo = p * q1
second_modulo = p * q2
first_exponent = 239
second_exponent = 17
first_ciphertext = Encrypt("attack", first_modulo, first_exponent)
second_ciphertext = Encrypt("wait", second_modulo, second_exponent)
print(DecipherCommonDivisor(first_ciphertext, first_modulo, first_exponent, second_ciphertext, second_modulo, second_exponent))


('attack', 'wait')


# Question 6, Hastad's Attack (Broadcast the same message using the same exponent)

In [8]:
def DecipherHastad(c1, n1, c2, n2, e):
  r = chineseRemainderTheorem(n1, c1, n2, c2)
  m = int(r**(e**-1))
  # m = int(math.sqrt(r)) # r = m^2 in case e = 2
  return ConvertToStr(m)

# question 
p1 = 790383132652258876190399065097
q1 = 662503581792812531719955475509
p2 = 656917682542437675078478868539
q2 = 1263581691331332127259083713503
n1 = p1 * q1
n2 = p2 * q2
e = 2
ciphertext1 = Encrypt("attack", n1, e)
ciphertext2 = Encrypt("attack", n2, e)
message = DecipherHastad(ciphertext1, n1, ciphertext2, n2, e)
print(message)

attack
